# Example: Exploring weather conditions around the recent L.A. fires with a single station

A demonstration of the capabilities of Synoptic's Weather API using the recent L.A. fires as an example. We'll begin by making a request for a single station to familiarize with the data structure, and then move to requesting data for multiple stations within an area of interest.

But first... we'll import **packages**

In [ ]:
import requests
import json
import pandas as pd

# Our main plotting package (must have explicit import of submodules)
import bokeh.io
import bokeh.plotting
# Enable viewing Bokeh plots in the notebook
bokeh.io.output_notebook()

## Step 1 -- Generate an API request for a single RAWS station in the vicinity of the Palisades fire

Any request requires a valid token. This one is not associated with the USFS account but can be freely used until it expires March 1.

In [ ]:
token = '07a920b581a1444a97ab4b722d6c9ed9'

We'll begin by requesting data for a single station in the vicinity of the Palisades Fire for a 5 month period of time beginning in Sept, 2024. This RAWS station reports air temperature, wind speed, wind gust, relative humidity, and fuel moisture. https://viewer.synopticdata.com/metadata/MBCC1/all/now

In [ ]:
base_url = 'https://api.synopticdata.com/v2/stations/'

timeseries_url = base_url+'timeseries?'
params = {
    'stids': 'MBCC1',
    'vars': 'fuel_moisture,air_temp,wind_speed,wind_gust,relative_humidity',
    'start': '202409010000',
    'end': '202501310000',
    'units': 'temp|F,speed|mph',
    'sensorvars': '1',
    'token': token,
}
r = requests.get(timeseries_url, params=params)

Check the API url and confirm successful return from inspection of the status: code

In [ ]:
print(f'API url: {r.url}')
print()
print(f'Status code: {r.status_code}')

API url: https://api.synopticdata.com/v2/stations/timeseries?stids=MBCC1&vars=fuel_moisture%2Cair_temp%2Cwind_speed%2Cwind_gust%2Crelative_humidity&start=202409010000&end=202501310000&units=temp%7CF%2Cspeed%7Cmph&sensorvars=1&token=07a920b581a1444a97ab4b722d6c9ed9

Status code: 200


##Step 2 -- Transform the response and explore the data structure
We can transform the response to data type that is easier to work with and confirm the data type and response summary:

In [ ]:
data = r.json()
print(type(data))
print()
print('API return payload keys')
print(list(data.keys()))

<class 'dict'>

API return payload keys
['STATION', 'SUMMARY', 'QC_SUMMARY', 'UNITS']


Some more general familiarization with the response output. Check the `SUMMARY` and `UNITS` keys.

In [ ]:
print('SUMMARY:')
print(json.dumps(data['SUMMARY'], indent=2))
print()
print('UNITS:')
print(json.dumps(data['UNITS'], indent=2))

SUMMARY:
{
  "NUMBER_OF_OBJECTS": 1,
  "RESPONSE_CODE": 1,
  "RESPONSE_MESSAGE": "OK",
  "METADATA_PARSE_TIME": "0.3 ms",
  "METADATA_DB_QUERY_TIME": "3.0 ms",
  "DATA_QUERY_TIME": "32.4 ms",
  "QC_QUERY_TIME": "4.9 ms",
  "DATA_PARSING_TIME": "70.0 ms",
  "TOTAL_DATA_TIME": "107.4 ms",
  "VERSION": "v2.25.1"
}

UNITS:
{
  "position": "ft",
  "elevation": "ft",
  "air_temp": "Fahrenheit",
  "relative_humidity": "%",
  "wind_speed": "Miles/hour",
  "wind_gust": "Miles/hour",
  "fuel_moisture": "gm"
}


We can see the `STATION` object is a list, composed of dictionaries. One dictionary for each station reporting. In this case we just have a single station. The station dictionary contains metadata for the station and the observations that we are interested in.

In [ ]:
print(type(data['STATION']))
print()
print('Station object keys:')
print(list(data['STATION'][0].keys()))

<class 'list'>

Station object keys:
['ID', 'STID', 'NAME', 'ELEVATION', 'LATITUDE', 'LONGITUDE', 'STATUS', 'MNET_ID', 'STATE', 'TIMEZONE', 'ELEV_DEM', 'PERIOD_OF_RECORD', 'SENSOR_VARIABLES', 'UNITS', 'OBSERVATIONS', 'QC_FLAGGED', 'RESTRICTED', 'RESTRICTED_METADATA']


The data structure has support for multiple datasets associated with a single variable type (e.g. soil temperature is often measured at multiple levels). This is captured in the station's `SENSOR_VARIABLES` object through a `variable_set_#` concept.

In [ ]:
print(json.dumps(data['STATION'][0]['SENSOR_VARIABLES'], indent=2))

{
  "air_temp": {
    "air_temp_set_1": {
      "position": "6.56",
      "PERIOD_OF_RECORD": {
        "start": "2005-09-28T00:00:00Z",
        "end": "2025-02-03T19:56:00Z"
      }
    }
  },
  "relative_humidity": {
    "relative_humidity_set_1": {
      "position": "6.56",
      "PERIOD_OF_RECORD": {
        "start": "2005-09-28T00:00:00Z",
        "end": "2025-02-03T19:56:00Z"
      }
    }
  },
  "wind_speed": {
    "wind_speed_set_1": {
      "position": "20.01",
      "PERIOD_OF_RECORD": {
        "start": "2005-09-28T00:00:00Z",
        "end": "2025-02-03T19:56:00Z"
      }
    }
  },
  "wind_gust": {
    "wind_gust_set_1": {
      "position": "20.01",
      "PERIOD_OF_RECORD": {
        "start": "2005-09-28T00:00:00Z",
        "end": "2025-02-03T19:56:00Z"
      }
    }
  },
  "fuel_moisture": {
    "fuel_moisture_set_1": {
      "position": "0.98",
      "PERIOD_OF_RECORD": {
        "start": "2005-09-28T00:00:00Z",
        "end": "2025-02-03T19:56:00Z"
      }
    }
  }
}


The `SENSOR_VARIABLES` object provides the mapping from the requested variables to different set numbers in the `OBSERVATIONS` dictionary.

In [ ]:
print('Station observations keys:')
print(list(data['STATION'][0]['OBSERVATIONS'].keys()))

Station observations keys:
['date_time', 'air_temp_set_1', 'relative_humidity_set_1', 'wind_speed_set_1', 'wind_gust_set_1', 'fuel_moisture_set_1']


##Step 3 -- convert the dictionary object to a data frame and plot
Now we can convert the dictionary object to a data frame for easy plotting

In [ ]:
df = pd.DataFrame(data['STATION'][0]['OBSERVATIONS'])
df['date_time'] = pd.to_datetime(df['date_time'], format='ISO8601')
df

,date_time,air_temp_set_1,relative_humidity_set_1,wind_speed_set_1,wind_gust_set_1,fuel_moisture_set_1
0,2024-09-01 00:56:00+00:00,77.0,60.0,17.0,32.00,7.7
1,2024-09-01 01:56:00+00:00,79.0,57.0,10.0,27.00,7.7
2,2024-09-01 02:56:00+00:00,71.0,72.0,5.0,14.99,7.8
3,2024-09-01 03:56:00+00:00,69.0,77.0,3.0,5.00,8.4
4,2024-09-01 04:56:00+00:00,67.0,81.0,1.0,3.00,9.2
...,...,...,...,...,...,...
3643,2025-01-30 19:56:00+00:00,54.0,76.0,4.0,9.00,18.8
3644,2025-01-30 20:56:00+00:00,53.0,78.0,7.0,14.01,15.3
3645,2025-01-30 21:56:00+00:00,55.0,70.0,4.0,13.00,14.6
3646,2025-01-30 22:56:00+00:00,56.0,63.0,7.0,11.00,13.0


And create basic plots of the data...

In [ ]:
# Create the figure, stored in variable `p`
var='wind_gust'
var_set = list(data['STATION'][0]['SENSOR_VARIABLES'][var].keys())[0]
p = bokeh.plotting.figure(
    width=700,
    height=500,
    x_axis_label='Date/time',
    y_axis_label=data['UNITS'][var],
    x_axis_type='datetime'
)
p.line(
    source=df,
    x='date_time',
    y=var_set
)
print(var_set)

wind_gust_set_1


In [ ]:
bokeh.io.show(p)

#Example 2: Exploring multiple stations

In [ ]:
params = {
    'radius': 'MBCC1,20',
    'vars': 'fuel_moisture,air_temp,wind_speed,wind_gust,relative_humidity',
    'start': '202501060000',
    'end': '202501090000',
    'units': 'temp|F,speed|mph',
    'token': token,
}
r2 = requests.get(timeseries_url, params=params)
data2 = r2.json()
data2['SUMMARY']

{'NUMBER_OF_OBJECTS': 394,
 'RESPONSE_CODE': 1,
 'RESPONSE_MESSAGE': 'OK',
 'METADATA_PARSE_TIME': '6.9 ms',
 'METADATA_DB_QUERY_TIME': '393.4 ms',
 'DATA_QUERY_TIME': '1126.0 ms',
 'QC_QUERY_TIME': '41.8 ms',
 'DATA_PARSING_TIME': '3220.9 ms',
 'TOTAL_DATA_TIME': '4044.4 ms',
 'VERSION': 'v2.25.1'}

In [ ]:
print(f"Verifying the number of stations returned in the request payload: {len(data2['STATION'])}")

Verifying the number of stations returned in the request payload: 394


Organize the data

In [ ]:
metadata_list = []
i = 0
for station in data2['STATION']:
  # Add metadata
  stid = station['STID']
  network_id = station['MNET_ID']
  try:
      lon = float(station['LONGITUDE'])
  except TypeError:
      lon = None
  try:
      lat = float(station['LATITUDE'])
  except TypeError:
      lat = None
  try:
      elev = float(station['ELEVATION'])
  except TypeError:
      elev = None
  metadata_list.append([stid, network_id, lon, lat, elev])
  # Add data
  df = pd.DataFrame()
  datetime = pd.to_datetime(station['OBSERVATIONS']['date_time'], format='ISO8601')
  multi_index = pd.MultiIndex.from_product([[station['STID']], datetime], names=['STID','date_time'])
  if i==0:
    data_df = pd.DataFrame(station['OBSERVATIONS'], index=multi_index)
  else:
    data_df = pd.concat([data_df, pd.DataFrame(station['OBSERVATIONS'], index=multi_index)], axis=0)
  i+=1

#Build metadata dataframe from list
meta_df = pd.DataFrame(metadata_list, columns=["stid", "mnet_id", "lon", "lat", "elev"])
meta_df.set_index('stid', inplace=True)

# Sort the resulting data dataframe by time
data_df.sort_index(inplace=True)
data_df.drop(['date_time'], axis=1, inplace=True)

In [ ]:
meta_df

,mnet_id,lon,lat,elev
stid,,,,
MBCC1,2,-118.70345,34.08394,610.00
584SE,231,-118.70491,34.07386,462.00
588SE,231,-118.70491,34.07388,462.00
DVI04930,3003,-118.69335,34.07758,595.00
SE204,231,-118.69172,34.07073,1279.00
...,...,...,...,...
213SE,231,-118.90411,34.31523,869.00
F9784,65,-118.39300,33.96100,157.00
SE418,231,-119.03780,34.15935,29.00


How many unique networks are represented here?

In [ ]:
print(f"Number of unique networks represented in payload: {len(meta_df['mnet_id'].unique())}")

Number of unique networks represented in payload: 19


Here's what the final data object looks like with multiple indices on station ID and date/time.


In [ ]:
data_df

air_temp_set_1  relative_humidity_set_1  \
STID  date_time                                                            
014SE 2025-01-06 00:00:00+00:00           64.05                    45.47   
      2025-01-06 00:10:00+00:00           63.73                    45.73   
      2025-01-06 00:20:00+00:00           63.37                    46.23   
      2025-01-06 00:30:00+00:00           62.94                    47.76   
      2025-01-06 00:40:00+00:00           62.32                    48.86   
...                                         ...                      ...   
WPCC1 2025-01-08 23:13:00+00:00           42.00                      NaN   
      2025-01-08 23:23:00+00:00           42.00                      NaN   
      2025-01-08 23:33:00+00:00           42.00                      NaN   
      2025-01-08 23:43:00+00:00           42.00                      NaN   
      2025-01-08 23:53:00+00:00           42.00                      NaN   

                                 wind_speed_set_1  wind_gust_set_1  \
STID  date_time                                                      
014SE 2025-01-06 00:00:00+00:00              1.99             4.23   
      2025-01-06 00:10:00+00:00              1.04             3.58   
      2025-01-06 00:20:00+00:00              2.02             3.73   
      2025-01-06 00:30:00+00:00              1.11             3.73   
      2025-01-06 00:40:00+00:00              1.02             2.99   
...                                           ...              ...   
WPCC1 2025-01-08 23:13:00+00:00               NaN              NaN   
      2025-01-08 23:23:00+00:00               NaN              NaN   
      2025-01-08 23:33:00+00:00               NaN              NaN   
      2025-01-08 23:43:00+00:00               NaN              NaN   
      2025-01-08 23:53:00+00:00               NaN              NaN   

                                 fuel_moisture_set_1  
STID  date_time                                       
014SE 2025-01-06 00:00:00+00:00                  NaN  
      2025-01-06 00:10:00+00:00                  NaN  
      2025-01-06 00:20:00+00:00                  NaN  
      2025-01-06 00:30:00+00:00                  NaN  
      2025-01-06 00:40:00+00:00                  NaN  
...                                              ...  
WPCC1 2025-01-08 23:13:00+00:00                  NaN  
      2025-01-08 23:23:00+00:00                  NaN  
      2025-01-08 23:33:00+00:00                  NaN  
      2025-01-08 23:43:00+00:00                  NaN  
      2025-01-08 23:53:00+00:00                  NaN  

[209220 rows x 5 columns]

We can do simple data exercises such as identifying the stations with top wind gusts over the 3 days requested:

In [ ]:
max_wind_gust = data_df['wind_gust_set_1'].groupby(level=0).max()
max_wind_gust.sort_values(ascending=False)[0:20]

,wind_gust_set_1
STID,
D2363,98.00
SE712,86.40
MBUC1,86.00
751SE,84.91
SE003,83.50
548SE,82.00
F0112,80.00
470SE,77.38
SE519,77.38


And, just for fun, look at max wind gust as a function of elevation across the stations.

In [ ]:
wind_elev = pd.concat([max_wind_gust, meta_df['elev']], axis=1)
wind_elev

,wind_gust_set_1,elev
014SE,31.35,207.00
043SE,30.25,810.00
070SE,48.74,1453.00
071SE,55.97,1453.00
072SE,45.59,515.00
...,...,...
WFT83317,19.01,272.65
WFTOP,51.90,0.00
WFWRB,73.10,0.00
WFZUM,59.90,0.00


In [ ]:
p2 = bokeh.plotting.figure(
    width=700,
    height=500,
    x_axis_label='Elevation (ft)',
    y_axis_label='Peak wind gust',
)

p2.scatter(
    source=wind_elev,
    x='elev',
    y='wind_gust_set_1'
)

bokeh.io.show(p2)